

---
# **Trabajo Final ECD 2022**

**Alumna:** Fiorella Cravero

**Director:** Dr. Axel Soto - Dr. Ignacio Ponzoni

**Título:** *Exploración* y evaluación de embeddings para la representación de polímeros basados en SMILES: comparativa con técnicas tradicionales de caracterización molecular


---




# **Instalación de librerias**

**Conetar con Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install mordred > /dev/null
!pip install rdkit > /dev/null
!pip install deepchem > /dev/null
!pip install git+https://github.com/samoturk/mol2vec > /dev/null
!pip install transformers > /dev/null

# **Representaciones tradicionales**

**Descriptores moleculares**

In [ ]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors

# Ruta del archivo CSV
archivo_csv = '/content/drive/MyDrive/TPfinalECD2022/polis_10000.csv'

# Carga el archivo CSV en un DataFrame de pandas
data = pd.read_csv(archivo_csv)

# Lista para almacenar los descriptores moleculares calculados
descriptores = []

# Itera sobre los SMILES y calcula los descriptores moleculares
for smile in data['SMILES']:
    mol = Chem.MolFromSmiles(smile)  # Convierte el SMILES en un objeto Mol
    if mol is not None:
        descriptor = {}  # Diccionario para almacenar los descriptores de cada molécula
        descriptor['SMILES'] = smile  # Agrega el SMILES al diccionario

        # Calcula todos los descriptores moleculares disponibles en RDKit
        for descriptor_name, descriptor_function in Descriptors.descList:
            descriptor[descriptor_name] = descriptor_function(mol)

        descriptores.append(descriptor)  # Agrega el diccionario a la lista de descriptores

# Crea un nuevo DataFrame a partir de la lista de descriptores
df_descriptores = pd.DataFrame(descriptores)

# Guarda los descriptores en un nuevo archivo CSV
ruta_guardar = '/content/drive/MyDrive/TPfinalECD2022/polis_10000_descriptores_molecularesOK.csv'
df_descriptores.to_csv(ruta_guardar, index=False)

print('Descriptores moleculares calculados y guardados en:', ruta_guardar)


**FingerPrints**

In [ ]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem

# Leer el archivo de polímeros
data = pd.read_csv('/content/drive/MyDrive/TPfinalECD2022/polis_100.csv')

# Crear una nueva columna para el fingerprint
data['Fingerprint'] = None

# Calcular el fingerprint para cada SMILES
for i, smiles in enumerate(data['SMILES']):
    mol = Chem.MolFromSmiles(smiles)
    if mol is not None:
        fingerprint = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=1024)
        data.at[i, 'Fingerprint'] = fingerprint.ToBitString()

# Guardar los resultados en un nuevo archivo
data.to_csv('/content/drive/MyDrive/TPfinalECD2022/polis_100_fingerprint.csv', index=False)



# **Embeddings**

**NO supervisado: Word2Vec**

In [2]:
#Word2Vec
import pandas as pd
import numpy as np
from gensim.models import Word2Vec

# Leer el archivo de polímeros
data = pd.read_csv('/content/drive/MyDrive/TPfinalECD2022/polis_30000.csv')

# Obtener los SMILES
smiles_list = data['SMILES'].tolist()

# Convertir los SMILES a listas de tokens
tokenized_smiles = [list(smiles) for smiles in smiles_list]

# Parámetros del modelo Word2Vec
vector_size = 10
# vector_size = 100
# vector_size = 1000
window = 5
min_count = 3
sg = 0

# Entrenar el modelo Word2Vec
model = Word2Vec(tokenized_smiles, vector_size=vector_size, window=window, min_count=min_count, sg=sg)

# Crear una lista para almacenar los embeddings y los SMILES que no se pueden calcular
embeddings = []
error_smiles = []

# Obtener los embeddings para cada polímero
for tokens in tokenized_smiles:
    embedding = np.zeros(vector_size)
    count = 0
    for token in tokens:
        if token in model.wv:
            embedding += model.wv[token]
            count += 1
    if count > 0:
        embeddings.append(embedding / count)
    else:
        embeddings.append(None)
        error_smiles.append(smiles)

# Crear un DataFrame con los embeddings
df = pd.DataFrame(embeddings, columns=[f'feature_{i}' for i in range(vector_size)])
df.to_csv(f'/content/drive/MyDrive/TPfinalECD2022/polis_30000_embeddings_Word2Vec_{vector_size}.csv', index=False)

# **Dimensionalidad de la representación**

**Para Fingerprints**

In [ ]:
import pandas as pd
import numpy as np

# Cargar el archivo CSV en un DataFrame
data = pd.read_csv('/content/drive/MyDrive/TPfinalECD2022/polis_30000_fingerprint.csv')

# Obtener la dimensionalidad de los números en la columna "Fingerprint"
data['Dimensionality'] = data['Fingerprint'].str.len()

# Obtener la cantidad de ceros en cada molécula
data['Zeros'] = data['Fingerprint'].apply(lambda x: x.count('0'))

# Calcular el promedio de ceros
promedio_ceros = data['Zeros'].mean()

# Imprimir el promedio de ceros por pantalla
print("El promedio de ceros es:", promedio_ceros)

# Escribir en el archivo CSV las columnas "SMILES", "Dimensionality" y "Zeros"
data[['SMILES', 'Dimensionality', 'Zeros']].to_csv('/content/drive/MyDrive/TPfinalECD2022/dimensionalidad_polis_30000_fingerprint.csv', index=False)

**Para Descriptores Moleculares**

In [ ]:
import pandas as pd

# Leer el archivo original
data = pd.read_csv('/content/drive/MyDrive/TPfinalECD2022/polis_10000_descriptores_molecularesOK.csv')

# Eliminar las columnas con valores faltantes en los descriptores
data_cleaned = data.dropna(axis=1, how='any')

# Obtener la dimensionalidad (cantidad de descriptores que quedaron)
dimensionality = len(data_cleaned.columns) - 1

# Calcular el promedio de ceros por columna
zero_counts = data_cleaned.iloc[:, 1:].eq(0).sum()
average_zeros = zero_counts.mean()

# Guardar el archivo modificado
data_cleaned.to_csv('/content/drive/MyDrive/TPfinalECD2022/polis_10000_descriptores_molecularesModificadoOK.csv', index=False)

# Imprimir la dimensionalidad y el promedio de ceros
print("Dimensionality:", dimensionality)
print("Average Zeros:", average_zeros)


**Para embeddings: Word2Vec**

In [ ]:
import pandas as pd

# Ruta del archivo CSV
archivo_csv = '/content/drive/MyDrive/TPfinalECD2022/polis_10000_embeddings_Word2Vec_100.csv'

# Leer el archivo CSV
dataframe = pd.read_csv(archivo_csv)

# Verificar si hay valores cero en el dataframe
valores_cero = dataframe[dataframe == 0].count().sum()

if valores_cero > 0:
    print("El archivo contiene valores cero.")
else:
    print("El archivo no contiene valores cero.")

# **Modelado predictivo**

# *Random Forest*

**Para MDs**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib

# Cargar los datos desde los archivos CSV
archivo_MDs = '/content/drive/MyDrive/TPfinalECD2022/polis_30000_descriptores_molecularesModificado.csv'
archivo_scores = '/content/drive/MyDrive/TPfinalECD2022/polis_30000.csv'

print("Ruta del archivo de MDs:", archivo_MDs)
print("Ruta del archivo de scores:", archivo_scores)

datos_MDs = pd.read_csv(archivo_MDs)
datos_scores = pd.read_csv(archivo_scores)

# Excluir la primera columna (SMILES) del archivo de MDs
datos_MDs = datos_MDs.iloc[:, 1:]

# Combinar los datos de embeddings y scores por columnas
datos = pd.concat([datos_MDs, datos_scores['SA Score']], axis=1)

# Extraer características (X) y etiquetas objetivo (y)
X = datos.drop('SA Score', axis=1).values
y = datos['SA Score'].values

# Normalizar las características dividiéndolas por el valor máximo
X = X / np.max(X)

# Dividir los datos en conjuntos de entrenamiento y prueba (70% entrenamiento, 30% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Definir los parámetros para la búsqueda en grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Crear y entrenar el modelo RandomForestRegressor utilizando GridSearchCV
modelo = RandomForestRegressor()
busqueda_grid = GridSearchCV(modelo, param_grid, cv=5)
busqueda_grid.fit(X_train, y_train)

# Obtener los mejores parámetros encontrados
mejores_parametros = busqueda_grid.best_params_
print("Mejores parámetros encontrados:", mejores_parametros)

# Crear y entrenar el modelo final utilizando los mejores parámetros
modelo = RandomForestRegressor(**mejores_parametros)
modelo.fit(X_train, y_train)

# Guardar el modelo entrenado en un archivo
joblib.dump(modelo, '/content/drive/MyDrive/TPfinalECD2022/modelo_entrenado_RF_polis_30000_descriptores_moleculares.pkl')

# Cargar el modelo entrenado desde el archivo
modelo_entrenado = joblib.load('/content/drive/MyDrive/TPfinalECD2022/modelo_entrenado_RF_polis_30000_descriptores_moleculares.pkl')

# Realizar predicciones en el conjunto de entrenamiento y prueba
y_train_pred = modelo_entrenado.predict(X_train)
y_test_pred = modelo_entrenado.predict(X_test)

# Calcular las métricas estadísticas
mse_train = mean_squared_error(y_train, y_train_pred)
mae_train = mean_absolute_error(y_train, y_train_pred)
r2_train = r2_score(y_train, y_train_pred)

mse_test = mean_squared_error(y_test, y_test_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)
r2_test = r2_score(y_test, y_test_pred)

# Crear un DataFrame con las métricas de entrenamiento y prueba
metricas = pd.DataFrame({
    'MSE_train': [mse_train],
    'MAE_train': [mae_train],
    'R2_train': [r2_train],
    'MSE_test': [mse_test],
    'MAE_test': [mae_test],
    'R2_test': [r2_test]
})

# Guardar las métricas en un archivo CSV
metricas.to_csv('/content/drive/MyDrive/TPfinalECD2022/metricas_RF_polis_polis_30000_descriptores_moleculares.csv', index=False)

# Imprimir las métricas de entrenamiento
print("Métricas del modelo de entrenamiento:")
print("MSE_train: ", mse_train)
print("MAE_train: ", mae_train)
print("R2_train: ", r2_train)

# Imprimir las métricas de prueba
print("Métricas del modelo de prueba:")
print("MSE_test: ", mse_test)
print("MAE_test: ", mae_test)
print("R2_test: ", r2_test)

# Imprimir DataFrame con las métricas
print('RF metricas:')
print(metricas)

**Para Fingerprints**

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.feature_extraction.text import CountVectorizer

# Cargar los datos desde el archivo CSV
data = pd.read_csv('/content/drive/MyDrive/TPfinalECD2022/polis_10000_fingerprint.csv')

# Separar las características (X) y la variable objetivo (y)
X = data['Fingerprint']
y = data['SA Score']

# Convertir los datos de texto a una representación numérica
vectorizer = CountVectorizer(binary=True)
X = vectorizer.fit_transform(X)

# Dividir los datos en conjuntos de entrenamiento y prueba (70% entrenamiento, 30% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Definir los parámetros a probar en la búsqueda de grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Crear y entrenar el modelo de regresión con Random Forest utilizando GridSearchCV
model = RandomForestRegressor()
grid_search = GridSearchCV(model, param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Obtener los mejores parámetros encontrados
mejores_parametros = grid_search.best_params_
print("Mejores parámetros encontrados:", mejores_parametros)

# Crear y entrenar el modelo final utilizando los mejores parámetros
modelo = RandomForestRegressor(**mejores_parametros)
modelo.fit(X_train, y_train)

# Guardar el modelo entrenado en un archivo
joblib.dump(modelo, '/content/drive/MyDrive/TPfinalECD2022/modelo_entrenado_RF_polis_10000_Fingerprints.pkl')

# Cargar el modelo entrenado desde el archivo
modelo_entrenado = joblib.load('/content/drive/MyDrive/TPfinalECD2022/modelo_entrenado_RF_polis_10000_Fingerprints.pkl')

# Realizar predicciones en el conjunto de entrenamiento y prueba
y_train_pred = modelo.predict(X_train)
y_test_pred = modelo.predict(X_test)

# Calcular las métricas estadísticas
mse_train = mean_squared_error(y_train, y_train_pred)
mae_train = mean_absolute_error(y_train, y_train_pred)
r2_train = r2_score(y_train, y_train_pred)

mse_test = mean_squared_error(y_test, y_test_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)
r2_test = r2_score(y_test, y_test_pred)

# Crear un DataFrame con las métricas de entrenamiento y prueba
metricas = pd.DataFrame({
    'MSE_train': [mse_train],
    'MAE_train': [mae_train],
    'R2_train': [r2_train],
    'MSE_test': [mse_test],
    'MAE_test': [mae_test],
    'R2_test': [r2_test]
})

# Guardar las métricas en un archivo CSV
metricas.to_csv('/content/drive/MyDrive/TPfinalECD2022/metricas_RF_polis_10000_fingerprints.csv', index=False)

# Imprimir las métricas de entrenamiento
print("Métricas del modelo de entrenamiento:")
print("MSE_train: ", mse_train)
print("MAE_train: ", mae_train)
print("R2_train: ", r2_train)

# Imprimir las métricas de prueba
print("Métricas del modelo de prueba:")
print("MSE_test: ", mse_test)
print("MAE_test: ", mae_test)
print("R2_test: ", r2_test)

# Imprimir DataFrame con las métricas
print('RF metricas:')
print(metricas)

**Para embeddings no sup (Word2Vec)**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib

# Cargar los datos desde los archivos CSV
archivo_embeddings = '/content/drive/MyDrive/TPfinalECD2022/polis_10000_embeddings_Word2Vec_1000.csv'
archivo_scores = '/content/drive/MyDrive/TPfinalECD2022/polis_10000.csv'

print("Ruta del archivo de embeddings:", archivo_embeddings)
print("Ruta del archivo de scores:", archivo_scores)

datos_embeddings = pd.read_csv(archivo_embeddings)
datos_scores = pd.read_csv(archivo_scores)

# Combinar los datos de embeddings y scores por columnas
datos = pd.concat([datos_embeddings, datos_scores['SA Score']], axis=1)

# Extraer características (X) y etiquetas objetivo (y)
X = datos.drop('SA Score', axis=1).values
y = datos['SA Score'].values

# Normalizar las características dividiéndolas por el valor máximo
X = X / np.max(X)

# Dividir los datos en conjuntos de entrenamiento y prueba (70% entrenamiento, 30% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Definir los parámetros para la búsqueda en grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Crear y entrenar el modelo RandomForestRegressor utilizando GridSearchCV
modelo = RandomForestRegressor()
busqueda_grid = GridSearchCV(modelo, param_grid, cv=5)
busqueda_grid.fit(X_train, y_train)

# Obtener los mejores parámetros encontrados
mejores_parametros = busqueda_grid.best_params_
print("Mejores parámetros encontrados:", mejores_parametros)

# Crear y entrenar el modelo final utilizando los mejores parámetros
modelo = RandomForestRegressor(**mejores_parametros)
modelo.fit(X_train, y_train)

# Guardar el modelo entrenado en un archivo
joblib.dump(modelo, '/content/drive/MyDrive/TPfinalECD2022/modelo_entrenado_RF_polis_10000_Word2Vec_1000.pkl')

# Cargar el modelo entrenado desde el archivo
modelo_entrenado = joblib.load('/content/drive/MyDrive/TPfinalECD2022/modelo_entrenado_RF_polis_10000_Word2Vec_1000.pkl')

# Realizar predicciones en el conjunto de entrenamiento y prueba
y_train_pred = modelo_entrenado.predict(X_train)
y_test_pred = modelo_entrenado.predict(X_test)

# Calcular las métricas estadísticas
mse_train = mean_squared_error(y_train, y_train_pred)
mae_train = mean_absolute_error(y_train, y_train_pred)
r2_train = r2_score(y_train, y_train_pred)

mse_test = mean_squared_error(y_test, y_test_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)
r2_test = r2_score(y_test, y_test_pred)

# Crear un DataFrame con las métricas de entrenamiento y prueba
metricas = pd.DataFrame({
    'MSE_train': [mse_train],
    'MAE_train': [mae_train],
    'R2_train': [r2_train],
    'MSE_test': [mse_test],
    'MAE_test': [mae_test],
    'R2_test': [r2_test]
})

# Guardar las métricas en un archivo CSV
metricas.to_csv('/content/drive/MyDrive/TPfinalECD2022/metricas_RF_polis_10000_embeddings_Word2Vec_1000.csv', index=False)

# Imprimir las métricas de entrenamiento
print("Métricas del modelo de entrenamiento:")
print("MSE_train: ", mse_train)
print("MAE_train: ", mae_train)
print("R2_train: ", r2_train)

# Imprimir las métricas de prueba
print("Métricas del modelo de prueba:")
print("MSE_test: ", mse_test)
print("MAE_test: ", mae_test)
print("R2_test: ", r2_test)

# Imprimir DataFrame con las métricas
print('RF metricas:')
print(metricas)



# *Regresion lineal*

**Para MDs**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib

# Cargar los datos desde los archivos CSV
archivo_MDs = '/content/drive/MyDrive/TPfinalECD2022/polis_10000_descriptores_molecularesModificadoOK.csv'
archivo_scores = '/content/drive/MyDrive/TPfinalECD2022/polis_10000.csv'

print("Ruta del archivo de MDs:", archivo_MDs)
print("Ruta del archivo de scores:", archivo_scores)

datos_MDs = pd.read_csv(archivo_MDs)
datos_scores = pd.read_csv(archivo_scores)

# Excluir la primera columna (SMILES) del archivo de MDs
datos_MDs = datos_MDs.iloc[:, 1:]

# Combinar los datos de embeddings y scores por columnas
datos = pd.concat([datos_MDs, datos_scores['SA Score']], axis=1)

# Extraer características (X) y etiquetas objetivo (y)
X = datos.drop('SA Score', axis=1).values
y = datos['SA Score'].values

# Normalizar las características dividiéndolas por el valor máximo
X = X / np.max(X)

# Dividir los datos en conjuntos de entrenamiento y prueba (70% entrenamiento, 30% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Crear y entrenar el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train, y_train)

# Guardar el modelo entrenado en un archivo
joblib.dump(modelo, '/content/drive/MyDrive/TPfinalECD2022/modelo_entrenado_LR_polis_10000_descriptores_molecularesOK.pkl')

# Cargar el modelo entrenado desde el archivo
modelo_entrenado = joblib.load('/content/drive/MyDrive/TPfinalECD2022/modelo_entrenado_LR_polis_10000_descriptores_molecularesOK.pkl')

# Realizar predicciones en el conjunto de entrenamiento y prueba
y_train_pred = modelo_entrenado.predict(X_train)
y_test_pred = modelo_entrenado.predict(X_test)

# Calcular las métricas estadísticas
mse_train = mean_squared_error(y_train, y_train_pred)
mae_train = mean_absolute_error(y_train, y_train_pred)
r2_train = r2_score(y_train, y_train_pred)

mse_test = mean_squared_error(y_test, y_test_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)
r2_test = r2_score(y_test, y_test_pred)

# Crear un DataFrame con las métricas de entrenamiento y prueba
metricas = pd.DataFrame({
    'MSE_train': [mse_train],
    'MAE_train': [mae_train],
    'R2_train': [r2_train],
    'MSE_test': [mse_test],
    'MAE_test': [mae_test],
    'R2_test': [r2_test]
})

# Guardar las métricas en un archivo CSV
metricas.to_csv('/content/drive/MyDrive/TPfinalECD2022/metricas_LR_polis_polis_10000_descriptores_molecularesOK.csv', index=False)

# Imprimir las métricas de entrenamiento
print("Métricas del modelo de entrenamiento:")
print("MSE_train:", mse_train)
print("MAE_train:", mae_train)
print("R2_train:", r2_train)

# Imprimir las métricas de prueba
print("Métricas del modelo de prueba:")
print("MSE_test:", mse_test)
print("MAE_test:", mae_test)
print("R2_test:", r2_test)

# Imprimir DataFrame con las métricas
print('Regresión Lineal métricas:')
print(metricas)


**Para Fingerprints**

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.feature_extraction.text import CountVectorizer

# Cargar los datos desde el archivo CSV
data = pd.read_csv('/content/drive/MyDrive/TPfinalECD2022/polis_30000_fingerprint.csv')

# Separar las características (X) y la variable objetivo (y)
X = data['Fingerprint']
y = data['SA Score']

# Convertir los datos de texto a una representación numérica
vectorizer = CountVectorizer(binary=True)
X = vectorizer.fit_transform(X)

# Dividir los datos en conjuntos de entrenamiento y prueba (70% entrenamiento, 30% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Crear y entrenar el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train, y_train)

# Guardar el modelo entrenado en un archivo
joblib.dump(modelo, '/content/drive/MyDrive/TPfinalECD2022/modelo_entrenado_LR_polis_30000_Fingerprints.pkl')

# Cargar el modelo entrenado desde el archivo
modelo_entrenado = joblib.load('/content/drive/MyDrive/TPfinalECD2022/modelo_entrenado_LR_polis_30000_Fingerprints.pkl')

# Realizar predicciones en el conjunto de entrenamiento y prueba
y_train_pred = modelo_entrenado.predict(X_train)
y_test_pred = modelo_entrenado.predict(X_test)

# Calcular las métricas estadísticas
mse_train = mean_squared_error(y_train, y_train_pred)
mae_train = mean_absolute_error(y_train, y_train_pred)
r2_train = r2_score(y_train, y_train_pred)

mse_test = mean_squared_error(y_test, y_test_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)
r2_test = r2_score(y_test, y_test_pred)

# Crear un DataFrame con las métricas de entrenamiento y prueba
metricas = pd.DataFrame({
    'MSE_train': [mse_train],
    'MAE_train': [mae_train],
    'R2_train': [r2_train],
    'MSE_test': [mse_test],
    'MAE_test': [mae_test],
    'R2_test': [r2_test]
})

# Guardar las métricas en un archivo CSV
metricas.to_csv('/content/drive/MyDrive/TPfinalECD2022/metricas_LR_polis_30000_fingerprints.csv', index=False)

# Imprimir las métricas de entrenamiento
print("Métricas del modelo de entrenamiento:")
print("MSE_train:", mse_train)
print("MAE_train:", mae_train)
print("R2_train:", r2_train)

# Imprimir las métricas de prueba
print("Métricas del modelo de prueba:")
print("MSE_test:", mse_test)
print("MAE_test:", mae_test)
print("R2_test:", r2_test)

# Imprimir DataFrame con las métricas
print('Regresión Lineal métricas:')
print(metricas)


**Para embeddings no sup (Word2Vec)**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib

# Cargar los datos desde los archivos CSV
archivo_embeddings = '/content/drive/MyDrive/TPfinalECD2022/polis_100_embeddings_Word2Vec_1000.csv'
archivo_scores = '/content/drive/MyDrive/TPfinalECD2022/polis_100.csv'

print("Ruta del archivo de embeddings:", archivo_embeddings)
print("Ruta del archivo de scores:", archivo_scores)

datos_embeddings = pd.read_csv(archivo_embeddings)
datos_scores = pd.read_csv(archivo_scores)

# Combinar los datos de embeddings y scores por columnas
datos = pd.concat([datos_embeddings, datos_scores['SA Score']], axis=1)

# Extraer características (X) y etiquetas objetivo (y)
X = datos.drop('SA Score', axis=1).values
y = datos['SA Score'].values

# Normalizar las características dividiéndolas por el valor máximo
X = X / np.max(X)

# Dividir los datos en conjuntos de entrenamiento y prueba (70% entrenamiento, 30% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Crear y entrenar el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train, y_train)

# Guardar el modelo entrenado en un archivo
joblib.dump(modelo, '/content/drive/MyDrive/TPfinalECD2022/modelo_entrenado_LR_polis_100_embeddings_Word2Vec_1000.pkl')

# Cargar el modelo entrenado desde el archivo
modelo_entrenado = joblib.load('/content/drive/MyDrive/TPfinalECD2022/modelo_entrenado_LR_polis_100_embeddings_Word2Vec_1000.pkl')


# Realizar predicciones en el conjunto de entrenamiento y prueba
y_train_pred = modelo.predict(X_train)
y_test_pred = modelo.predict(X_test)

# Calcular las métricas estadísticas
mse_train = mean_squared_error(y_train, y_train_pred)
mae_train = mean_absolute_error(y_train, y_train_pred)
r2_train = r2_score(y_train, y_train_pred)

mse_test = mean_squared_error(y_test, y_test_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)
r2_test = r2_score(y_test, y_test_pred)

# Crear un DataFrame con las métricas de entrenamiento y prueba
metricas = pd.DataFrame({
    'MSE_train': [mse_train],
    'MAE_train': [mae_train],
    'R2_train': [r2_train],
    'MSE_test': [mse_test],
    'MAE_test': [mae_test],
    'R2_test': [r2_test]
})

# Guardar las métricas en un archivo CSV
metricas.to_csv('/content/drive/MyDrive/TPfinalECD2022/metricas_LR_polis_100_embeddings_Word2Vec_1000.csv', index=False)

# Imprimir las métricas de entrenamiento
print("Métricas del modelo de entrenamiento:")
print("MSE_train:", mse_train)
print("MAE_train:", mae_train)
print("R2_train:", r2_train)

# Imprimir las métricas de prueba
print("Métricas del modelo de prueba:")
print("MSE_test:", mse_test)
print("MAE_test:", mae_test)
print("R2_test:", r2_test)

# Imprimir DataFrame con las métricas
print('Regresión Lineal métricas:')
print(metricas)


# *Dummy Regression*

**Para MDs**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib

# Cargar los datos desde los archivos CSV
archivo_MDs = '/content/drive/MyDrive/TPfinalECD2022/polis_30000_descriptores_molecularesModificado.csv'
archivo_scores = '/content/drive/MyDrive/TPfinalECD2022/polis_30000.csv'

print("Ruta del archivo de MDs:", archivo_MDs)
print("Ruta del archivo de scores:", archivo_scores)

datos_MDs = pd.read_csv(archivo_MDs)
datos_scores = pd.read_csv(archivo_scores)

# Excluir la primera columna (SMILES) del archivo de MDs
datos_MDs = datos_MDs.iloc[:, 1:]

# Combinar los datos de embeddings y scores por columnas
datos = pd.concat([datos_MDs, datos_scores['SA Score']], axis=1)

# Extraer características (X) y etiquetas objetivo (y)
X = datos.drop('SA Score', axis=1).values
y = datos['SA Score'].values

# Normalizar las características dividiéndolas por el valor máximo
X = X / np.max(X)

# Dividir los datos en conjuntos de entrenamiento y prueba (70% entrenamiento, 30% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Crear y entrenar el modelo de regresión dummy
modelo = DummyRegressor(strategy='mean')
modelo.fit(X_train, y_train)

# Guardar el modelo entrenado en un archivo
joblib.dump(modelo, '/content/drive/MyDrive/TPfinalECD2022/modelo_entrenado_dummy_polis_30000_descriptores_moleculares.pkl')

# Cargar el modelo entrenado desde el archivo
modelo_entrenado = joblib.load('/content/drive/MyDrive/TPfinalECD2022/modelo_entrenado_dummy_polis_30000_descriptores_moleculares.pkl')

# Realizar predicciones en el conjunto de entrenamiento y prueba
y_train_pred = modelo_entrenado.predict(X_train)
y_test_pred = modelo_entrenado.predict(X_test)

# Calcular las métricas estadísticas
mse_train = mean_squared_error(y_train, y_train_pred)
mae_train = mean_absolute_error(y_train, y_train_pred)
r2_train = r2_score(y_train, y_train_pred)

mse_test = mean_squared_error(y_test, y_test_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)
r2_test = r2_score(y_test, y_test_pred)

# Crear un DataFrame con las métricas de entrenamiento y prueba
metricas = pd.DataFrame({
    'MSE_train': [mse_train],
    'MAE_train': [mae_train],
    'R2_train': [r2_train],
    'MSE_test': [mse_test],
    'MAE_test': [mae_test],
    'R2_test': [r2_test]
})

# Guardar las métricas en un archivo CSV
metricas.to_csv('/content/drive/MyDrive/TPfinalECD2022/metricas_dummy_polis_polis_30000_descriptores_moleculares.csv', index=False)

# Imprimir las métricas de entrenamiento
print("Métricas del modelo de entrenamiento:")
print("MSE_train:", mse_train)
print("MAE_train:", mae_train)
print("R2_train:", r2_train)

# Imprimir las métricas de prueba
print("Métricas del modelo de prueba:")
print("MSE_test:", mse_test)
print("MAE_test:", mae_test)
print("R2_test:", r2_test)

# Imprimir DataFrame con las métricas
print('Dummy Regression métricas:')
print(metricas)


Ruta del archivo de MDs: /content/drive/MyDrive/TPfinalECD2022/polis_30000_descriptores_molecularesModificado.csv
Ruta del archivo de scores: /content/drive/MyDrive/TPfinalECD2022/polis_30000.csv
Métricas del modelo de entrenamiento:
MSE_train: 0.7064402349542779
MAE_train: 0.6542320125930519
R2_train: 0.0
Métricas del modelo de prueba:
MSE_test: 0.7233041692728642
MAE_test: 0.6601170438720777
R2_test: -0.0007988530875020494
Dummy Regression métricas:
   MSE_train  MAE_train  R2_train  MSE_test  MAE_test   R2_test
0    0.70644   0.654232       0.0  0.723304  0.660117 -0.000799


**Para Fingerprints**

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.feature_extraction.text import CountVectorizer

# Cargar los datos desde el archivo CSV
data = pd.read_csv('/content/drive/MyDrive/TPfinalECD2022/polis_100_fingerprint.csv')

# Separar las características (X) y la variable objetivo (y)
X = data['Fingerprint']
y = data['SA Score']

# Convertir los datos de texto a una representación numérica
vectorizer = CountVectorizer(binary=True)
X = vectorizer.fit_transform(X)

# Dividir los datos en conjuntos de entrenamiento y prueba (70% entrenamiento, 30% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Crear y entrenar el modelo de regresión dummy
modelo = DummyRegressor(strategy='mean')
modelo.fit(X_train, y_train)

# Guardar el modelo entrenado en un archivo
joblib.dump(modelo, '/content/drive/MyDrive/TPfinalECD2022/modelo_entrenado_dummy_polis_100_Fingerprints.pkl')

# Cargar el modelo entrenado desde el archivo
modelo_entrenado = joblib.load('/content/drive/MyDrive/TPfinalECD2022/modelo_entrenado_dummy_polis_100_Fingerprints.pkl')

# Realizar predicciones en el conjunto de entrenamiento y prueba
y_train_pred = modelo_entrenado.predict(X_train)
y_test_pred = modelo_entrenado.predict(X_test)

# Calcular las métricas estadísticas
mse_train = mean_squared_error(y_train, y_train_pred)
mae_train = mean_absolute_error(y_train, y_train_pred)
r2_train = r2_score(y_train, y_train_pred)

mse_test = mean_squared_error(y_test, y_test_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)
r2_test = r2_score(y_test, y_test_pred)

# Crear un DataFrame con las métricas de entrenamiento y prueba
metricas = pd.DataFrame({
    'MSE_train': [mse_train],
    'MAE_train': [mae_train],
    'R2_train': [r2_train],
    'MSE_test': [mse_test],
    'MAE_test': [mae_test],
    'R2_test': [r2_test]
})

# Guardar las métricas en un archivo CSV
metricas.to_csv('/content/drive/MyDrive/TPfinalECD2022/metricas_dummy_polis_100_fingerprints.csv', index=False)

# Imprimir las métricas de entrenamiento
print("Métricas del modelo de entrenamiento:")
print("MSE_train:", mse_train)
print("MAE_train:", mae_train)
print("R2_train:", r2_train)

# Imprimir las métricas de prueba
print("Métricas del modelo de prueba:")
print("MSE_test:", mse_test)
print("MAE_test:", mae_test)
print("R2_test:", r2_test)

# Imprimir DataFrame con las métricas
print('Regresión Lineal métricas:')
print(metricas)


Métricas del modelo de entrenamiento:
MSE_train: 1.1877657607935164
MAE_train: 0.9250672284864978
R2_train: 0.0
Métricas del modelo de prueba:
MSE_test: 0.7553775591683376
MAE_test: 0.7484907421785972
R2_test: -0.20644651413342086
Regresión Lineal métricas:
   MSE_train  MAE_train  R2_train  MSE_test  MAE_test   R2_test
0   1.187766   0.925067       0.0  0.755378  0.748491 -0.206447


**Para embeddings no sup (Word2Vec)**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib

# Cargar los datos desde los archivos CSV
archivo_embeddings = '/content/drive/MyDrive/TPfinalECD2022/polis_1000_embeddings_Word2Vec_10.csv'
archivo_scores = '/content/drive/MyDrive/TPfinalECD2022/polis_1000.csv'

print("Ruta del archivo de embeddings:", archivo_embeddings)
print("Ruta del archivo de scores:", archivo_scores)

datos_embeddings = pd.read_csv(archivo_embeddings)
datos_scores = pd.read_csv(archivo_scores)

# Combinar los datos de embeddings y scores por columnas
datos = pd.concat([datos_embeddings, datos_scores['SA Score']], axis=1)

# Extraer características (X) y etiquetas objetivo (y)
X = datos.drop('SA Score', axis=1).values
y = datos['SA Score'].values

# Normalizar las características dividiéndolas por el valor máximo
X = X / np.max(X)

# Dividir los datos en conjuntos de entrenamiento y prueba (70% entrenamiento, 30% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Crear y entrenar el modelo de regresión dummy
modelo = DummyRegressor(strategy='mean')
modelo.fit(X_train, y_train)

# Guardar el modelo entrenado en un archivo
joblib.dump(modelo, '/content/drive/MyDrive/TPfinalECD2022/modelo_entrenado_dummy_polis_1000_embeddings_Word2Vec_10.pkl')

# Cargar el modelo entrenado desde el archivo
modelo_entrenado = joblib.load('/content/drive/MyDrive/TPfinalECD2022/modelo_entrenado_dummy_polis_1000_embeddings_Word2Vec_10.pkl')


# Realizar predicciones en el conjunto de entrenamiento y prueba
y_train_pred = modelo.predict(X_train)
y_test_pred = modelo.predict(X_test)

# Calcular las métricas estadísticas
mse_train = mean_squared_error(y_train, y_train_pred)
mae_train = mean_absolute_error(y_train, y_train_pred)
r2_train = r2_score(y_train, y_train_pred)

mse_test = mean_squared_error(y_test, y_test_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)
r2_test = r2_score(y_test, y_test_pred)

# Crear un DataFrame con las métricas de entrenamiento y prueba
metricas = pd.DataFrame({
    'MSE_train': [mse_train],
    'MAE_train': [mae_train],
    'R2_train': [r2_train],
    'MSE_test': [mse_test],
    'MAE_test': [mae_test],
    'R2_test': [r2_test]
})

# Guardar las métricas en un archivo CSV
metricas.to_csv('/content/drive/MyDrive/TPfinalECD2022/metricas_dummy_polis_1000_embeddings_Word2Vec_10.csv', index=False)

# Imprimir las métricas de entrenamiento
print("Métricas del modelo de entrenamiento:")
print("MSE_train:", mse_train)
print("MAE_train:", mae_train)
print("R2_train:", r2_train)

# Imprimir las métricas de prueba
print("Métricas del modelo de prueba:")
print("MSE_test:", mse_test)
print("MAE_test:", mae_test)
print("R2_test:", r2_test)

# Imprimir DataFrame con las métricas
print('Regresión Lineal métricas:')
print(metricas)


Ruta del archivo de embeddings: /content/drive/MyDrive/TPfinalECD2022/polis_1000_embeddings_Word2Vec_10.csv
Ruta del archivo de scores: /content/drive/MyDrive/TPfinalECD2022/polis_1000.csv
Métricas del modelo de entrenamiento:
MSE_train: 0.8465870975945702
MAE_train: 0.7468155484330873
R2_train: 0.0
Métricas del modelo de prueba:
MSE_test: 0.9552140611756611
MAE_test: 0.7773608852701035
R2_test: -0.001406481821576211
Regresión Lineal métricas:
   MSE_train  MAE_train  R2_train  MSE_test  MAE_test   R2_test
0   0.846587   0.746816       0.0  0.955214  0.777361 -0.001406


# **Análisis de valores atípicos para regresión lineal con DMs**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib
from rdkit import Chem
from rdkit.Chem import Descriptors

# Ruta del archivo CSV
archivo_csv = '/content/drive/MyDrive/TPfinalECD2022/polis_10000.csv'

# Carga el archivo CSV en un DataFrame de pandas
data = pd.read_csv(archivo_csv)

# Lista para almacenar los descriptores moleculares calculados
descriptores = []

# Itera sobre los SMILES y calcula los descriptores moleculares
for smile in data['SMILES']:
    mol = Chem.MolFromSmiles(smile)  # Convierte el SMILES en un objeto Mol
    if mol is not None:
        descriptor = {}  # Diccionario para almacenar los descriptores de cada molécula
        descriptor['SMILES'] = smile  # Agrega el SMILES al diccionario

        # Calcula todos los descriptores moleculares disponibles en RDKit
        for descriptor_name, descriptor_function in Descriptors.descList:
            descriptor[descriptor_name] = descriptor_function(mol)

        descriptores.append(descriptor)  # Agrega el diccionario a la lista de descriptores

# Crea un nuevo DataFrame a partir de la lista de descriptores
df_descriptores = pd.DataFrame(descriptores)

# Guarda los descriptores en un nuevo archivo CSV
ruta_guardar = '/content/drive/MyDrive/TPfinalECD2022/AnalisisValoresAtípicos_polis_10000_DMs.csv'
df_descriptores.to_csv(ruta_guardar, index=False)

datos_MDs = pd.read_csv('/content/drive/MyDrive/TPfinalECD2022/AnalisisValoresAtípicos_polis_10000_DMs.csv')
datos_scores = pd.read_csv('/content/drive/MyDrive/TPfinalECD2022/polis_10000.csv')

# Eliminar las columnas con valores faltantes en los descriptores
datos_MDs = datos_MDs.dropna(axis=1, how='any')

# Excluir la primera columna (SMILES) del archivo de MDs
datos_MDs = datos_MDs.iloc[:, 1:]

# Combinar los datos de embeddings y scores por columnas
datos = pd.concat([datos_MDs, datos_scores['SA Score']], axis=1)

# Extraer características (X) y etiquetas objetivo (y)
X = datos.drop('SA Score', axis=1).values
y = datos['SA Score'].values

# Normalizar las características dividiéndolas por el valor máximo
X = X / np.max(X)

# Dividir los datos en conjuntos de entrenamiento y prueba (70% entrenamiento, 30% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Crear y entrenar el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train, y_train)

# Realizar predicciones en el conjunto de entrenamiento y prueba
y_train_pred = modelo.predict(X_train)
y_test_pred = modelo.predict(X_test)

# Guardar los valores reales y predichos del conjunto de entrenamiento en un archivo CSV
df_train = pd.DataFrame({'Real': y_train, 'Predicho': y_train_pred})
df_train.to_csv('/content/drive/MyDrive/TPfinalECD2022/ValoresAtipicosMD10000_train.csv', index=False)

# Guardar los valores reales y predichos del conjunto de prueba en un archivo CSV
df_test = pd.DataFrame({'Real': y_test, 'Predicho': y_test_pred})
df_test.to_csv('/content/drive/MyDrive/TPfinalECD2022/ValoresAtipicosMD10000_test.csv', index=False)

# Graficar los valores reales versus los valores predichos
plt.scatter(y_train, y_train_pred, color='red', label='Train')
plt.scatter(y_test, y_test_pred, color='cyan', label='Test')
plt.xlabel('Valores Reales')
plt.ylabel('Valores Predichos')
plt.title('Valores Reales vs. Valores Predichos')
plt.legend()
plt.show()

# Calcular las métricas de evaluación
mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)
mae_train = mean_absolute_error(y_train, y_train_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)
r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)

# Imprimir las métricas de evaluación
print("MSE - Train: ", mse_train)
print("MSE - Test: ", mse_test)
print("MAE - Train: ", mae_train)
print("MAE - Test: ", mae_test)
print("R^2 - Train: ", r2_train)
print("R^2 - Test: ", r2_test)

# Calcular el error absoluto
absolute_errors_train = np.abs(y_train - y_train_pred)
absolute_errors_test = np.abs(y_test - y_test_pred)

# Calcular el error promedio
mean_error_train = np.mean(absolute_errors_train)
mean_error_test = np.mean(absolute_errors_test)

# Calcular el umbral para el error
threshold_test = mean_error_test * 10

# Encontrar los índices de las moléculas con error mayor al umbral en el conjunto de entrenamiento
high_error_indices_train = np.where(absolute_errors_train > threshold_train)[0]

# Encontrar los índices de las moléculas con error mayor al umbral en el conjunto de prueba
high_error_indices_test = np.where(absolute_errors_test > threshold_test)[0]

# Imprimir información de las moléculas con error mayor al umbral en el conjunto de prueba
print("Moléculas en el conjunto de prueba con error 10 veces mayor que el promedio:")
for index in high_error_indices_test:
    print("Número de fila:", index)
    print("Valor real:", y_test[index])
    print("Valor predicho:", y_test_pred[index])
    print("Error:", absolute_errors_test[index])
    print()

# **Modelado con Random Forest sin optimización de hiperparámtros**

In [ ]:
# Para Fingerprints

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.feature_extraction.text import CountVectorizer
import joblib

# Cargar los datos desde el archivo CSV
data = pd.read_csv('/content/drive/MyDrive/TPfinalECD2022/polis_30000_fingerprint.csv')

# Separar las características (X) y la variable objetivo (y)
X = data['Fingerprint']
y = data['SA Score']

# Convertir los datos de texto a una representación numérica
vectorizer = CountVectorizer(binary=True)
X = vectorizer.fit_transform(X)

# Dividir los datos en conjuntos de entrenamiento y prueba (70% entrenamiento, 30% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Crear y entrenar el modelo de regresión con Random Forest
model = RandomForestRegressor(n_estimators=200, max_depth=5, min_samples_split=2, min_samples_leaf=1)
model.fit(X_train, y_train)

# Guardar el modelo entrenado en un archivo
joblib.dump(model, '/content/drive/MyDrive/TPfinalECD2022/NoOptim_modelo_RF_polis_30000_Fingerprints.pkl')

# Cargar el modelo entrenado desde el archivo
modelo_entrenado = joblib.load('/content/drive/MyDrive/TPfinalECD2022/NoOptim_modelo_RF_polis_30000_Fingerprints.pkl')

# Realizar predicciones en el conjunto de entrenamiento y prueba
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Calcular las métricas estadísticas
mse_train = mean_squared_error(y_train, y_train_pred)
mae_train = mean_absolute_error(y_train, y_train_pred)
r2_train = r2_score(y_train, y_train_pred)

mse_test = mean_squared_error(y_test, y_test_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)
r2_test = r2_score(y_test, y_test_pred)

# Crear un DataFrame con las métricas de entrenamiento y prueba
metricas = pd.DataFrame({
    'MSE_train': [mse_train],
    'MAE_train': [mae_train],
    'R2_train': [r2_train],
    'MSE_test': [mse_test],
    'MAE_test': [mae_test],
    'R2_test': [r2_test]
})

# Guardar las métricas en un archivo CSV
metricas.to_csv('/content/drive/MyDrive/TPfinalECD2022/NoOptim_metricas_RF_polis_30000_fingerprints.csv', index=False)

# Imprimir las métricas de entrenamiento
print("Métricas del modelo de entrenamiento:")
print("MSE_train: ", mse_train)
print("MAE_train: ", mae_train)
print("R2_train: ", r2_train)

# Imprimir las métricas de prueba
print("Métricas del modelo de prueba:")
print("MSE_test: ", mse_test)
print("MAE_test: ", mae_test)
print("R2_test: ", r2_test)

# Imprimir DataFrame con las métricas
print('RF metricas:')
print(metricas)

In [ ]:
# Para embeddings no sup (Word2Vec)

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib

# Cargar los datos desde los archivos CSV
archivo_embeddings = '/content/drive/MyDrive/TPfinalECD2022/polis_10000_embeddings_Word2Vec_1000.csv'
archivo_scores = '/content/drive/MyDrive/TPfinalECD2022/polis_10000.csv'

print("Ruta del archivo de embeddings:", archivo_embeddings)
print("Ruta del archivo de scores:", archivo_scores)

datos_embeddings = pd.read_csv(archivo_embeddings)
datos_scores = pd.read_csv(archivo_scores)

# Combinar los datos de embeddings y scores por columnas
datos = pd.concat([datos_embeddings, datos_scores['SA Score']], axis=1)

# Extraer características (X) y etiquetas objetivo (y)
X = datos.drop('SA Score', axis=1).values
y = datos['SA Score'].values

# Normalizar las características dividiéndolas por el valor máximo
X = X / np.max(X)

# Dividir los datos en conjuntos de entrenamiento y prueba (70% entrenamiento, 30% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Crear y entrenar el modelo de regresión con Random Forest
model = RandomForestRegressor(n_estimators=200, max_depth=5, min_samples_split=2, min_samples_leaf=1)
model.fit(X_train, y_train)

# Guardar el modelo entrenado en un archivo
joblib.dump(model, '/content/drive/MyDrive/TPfinalECD2022/NoOptim_modelo_RF_polis_10000_Word2Vec_1000.pkl')

# Cargar el modelo entrenado desde el archivo
modelo_entrenado = joblib.load('/content/drive/MyDrive/TPfinalECD2022/NoOptim_modelo_RF_polis_10000_Word2Vec_1000.pkl')

# Realizar predicciones en el conjunto de entrenamiento y prueba
y_train_pred = modelo_entrenado.predict(X_train)
y_test_pred = modelo_entrenado.predict(X_test)

# Calcular las métricas estadísticas
mse_train = mean_squared_error(y_train, y_train_pred)
mae_train = mean_absolute_error(y_train, y_train_pred)
r2_train = r2_score(y_train, y_train_pred)

mse_test = mean_squared_error(y_test, y_test_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)
r2_test = r2_score(y_test, y_test_pred)

# Crear un DataFrame con las métricas de entrenamiento y prueba
metricas = pd.DataFrame({
    'MSE_train': [mse_train],
    'MAE_train': [mae_train],
    'R2_train': [r2_train],
    'MSE_test': [mse_test],
    'MAE_test': [mae_test],
    'R2_test': [r2_test]
})

# Guardar las métricas en un archivo CSV
metricas.to_csv('/content/drive/MyDrive/TPfinalECD2022/NoOptim_metricas_RF_polis_10000_embeddings_Word2Vec_1000.csv', index=False)

# Imprimir las métricas de entrenamiento
print("Métricas del modelo de entrenamiento:")
print("MSE_train: ", mse_train)
print("MAE_train: ", mae_train)
print("R2_train: ", r2_train)

# Imprimir las métricas de prueba
print("Métricas del modelo de prueba:")
print("MSE_test: ", mse_test)
print("MAE_test: ", mae_test)
print("R2_test: ", r2_test)

# Imprimir DataFrame con las métricas
print('RF metricas:')
print(metricas)